In [85]:
#Gross Vehicle Weight Analysis of WIM Data

Criteria for compliant / overweight by class

Gross Vehicle Weights By Class

In [86]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Capstone_Data") \
    .master("local[*]") \
    .getOrCreate()
print("Spark is running. Don't forget to stop session to terminate background java process!")


Spark is running. Don't forget to stop session to terminate background java process!


Historical (2019-2023)

In [87]:
df = spark.read.csv("Data\Raw_Data\Weigh_in_Motion_19_23.csv", header=True, inferSchema=True)
df.show(5)

<>:1: SyntaxWarning: invalid escape sequence '\R'
<>:1: SyntaxWarning: invalid escape sequence '\R'
C:\Users\rubie\AppData\Local\Temp\ipykernel_9804\1412014802.py:1: SyntaxWarning: invalid escape sequence '\R'
  df = spark.read.csv("Data\Raw_Data\Weigh_in_Motion_19_23.csv", header=True, inferSchema=True)


+---------+----+-----+---+----+------+------+----+-----+-----+---------+----+---------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+
|direction|year|month|day|hour|minute|second|lane|class|speed|num_axles| gvw|wheelbase| wt1|spc1| wt2|spc2| wt3|spc3| wt4|spc4| wt5|spc5| wt6|spc6| wt7|spc7| wt8|spc8| wt9|spc9|wt10|spc10|wt11|spc11|wt12|spc12|wt13|spc13|wt14|spc14|wt15|spc15|wt16|spc16|wt17|dirtravel|yearmonth|
+---------+----+-----+---+----+------+------+----+-----+-----+---------+----+---------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+
|       QB|  23|    5|  8|  17|    53|    16|   2|    2|   49|        2|4880|      9.5|2700| 9.5|2180|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NUL

In [88]:
df.createOrReplaceTempView("table1")

In [89]:
#New column "exceed_80000" 1->exceeds 0-> doesnt, check if GVW exceed 80,000 lbs
#If gvw 10% above limit then ticket is issued , marked as 1 in Ticket col
df2 = spark.sql(
    f"""
        SELECT *, 
            CASE
                WHEN gvw > 80000 THEN 1
                ELSE 0
            END AS exceed_80000,
            CASE
                WHEN gvw > 88000 THEN 1
                ELSE 0
            END AS Ticket
    FROM table1
    WHERE direction = 'QB'
    """
)

df2.show(5)

+---------+----+-----+---+----+------+------+----+-----+-----+---------+----+---------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------+
|direction|year|month|day|hour|minute|second|lane|class|speed|num_axles| gvw|wheelbase| wt1|spc1| wt2|spc2| wt3|spc3| wt4|spc4| wt5|spc5| wt6|spc6| wt7|spc7| wt8|spc8| wt9|spc9|wt10|spc10|wt11|spc11|wt12|spc12|wt13|spc13|wt14|spc14|wt15|spc15|wt16|spc16|wt17|dirtravel|yearmonth|exceed_80000|Ticket|
+---------+----+-----+---+----+------+------+----+-----+-----+---------+----+---------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------+
|       QB|  23|    5|  8|  17|    53|    16|   2|    2|   49|        2|4880|      9.5|2700| 9.5|218

In [90]:
exceed_gvw_limit_19_23 = df2.filter("exceed_80000 ==  1").count()

exceed_gvw_limit_19_23
#461,660 exceed 80,000 lb limit from 2019-2023

461660

In [91]:
exceed_gvw_limit_19_23_ticket_count = df2.filter("Ticket ==  1").count()

exceed_gvw_limit_19_23_ticket_count
#205,233 Tickets issued for GVW > 88000 lb

205233

In [92]:
df2.createOrReplaceTempView("table2")

In [93]:
#If any single axle load is > 22400 lb weight is non-compliant marked as 1 in col: exceed_22400
#If any single axle load is > 24,640 lb col Ticket = 1

df3 = spark.sql(
    """ 
        SELECT * EXCEPT (Ticket), 
            CASE
                WHEN GREATEST(
                    wt1, wt2, wt3, wt4, wt5, wt6, wt7, wt8, 
                    wt9, wt10, wt11, wt12, wt13, wt14, wt15, wt16, wt17
                ) > 22400 THEN 1
                ELSE 0
            END AS exceed_22400,
            CASE
                WHEN GREATEST(
                    wt1, wt2, wt3, wt4, wt5, wt6, wt7, wt8, 
                    wt9, wt10, wt11, wt12, wt13, wt14, wt15, wt16, wt17
                ) > 24640 THEN 1
                ELSE Ticket  
            END AS Ticket
        FROM table2
        WHERE direction = 'QB'
    """
)

df3.show(5)

+---------+----+-----+---+----+------+------+----+-----+-----+---------+----+---------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------------+------+
|direction|year|month|day|hour|minute|second|lane|class|speed|num_axles| gvw|wheelbase| wt1|spc1| wt2|spc2| wt3|spc3| wt4|spc4| wt5|spc5| wt6|spc6| wt7|spc7| wt8|spc8| wt9|spc9|wt10|spc10|wt11|spc11|wt12|spc12|wt13|spc13|wt14|spc14|wt15|spc15|wt16|spc16|wt17|dirtravel|yearmonth|exceed_80000|exceed_22400|Ticket|
+---------+----+-----+---+----+------+------+----+-----+-----+---------+----+---------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------------+------+
|       QB|  23|    5|  8|  17|    53|    16|   2|    2|   49

In [94]:
exceed_1axle_limit_19_23 = df3.filter("exceed_22400 ==  1").count()

exceed_1axle_limit_19_23
#250,157 exceed 22400 lb one axle wt limit from 2019-2023

250157

In [96]:
exceed_1axle_limit_19_23_ticket_count = df3.filter("Ticket ==  1").count() - exceed_gvw_limit_19_23_ticket_count 

exceed_1axle_limit_19_23_ticket_count
#205,233 Tickets issued for GVW > 88000 lb
#89,659 Tickets issued for single axle overload

89659

In [97]:
df3.createOrReplaceTempView("table3")

In [98]:
#data spacing units are in meters
#Spacing requirement made for any two axles with  54" < spacing < 120"  approx 1.37 meters < s < 3.05 meters

spc_conditions = " OR ".join([f"(spc{i} >= 1.37 AND spc{i} <= 3.05)" for i in range(1, 17)])

df4 = spark.sql(f"""
    SELECT *,
        CASE 
            WHEN {spc_conditions} THEN 1 
            ELSE 0 
        END AS check_spacing
    FROM table3
    WHERE direction = 'QB'
""")

df4.show(5)


+---------+----+-----+---+----+------+------+----+-----+-----+---------+----+---------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------------+------+-------------+
|direction|year|month|day|hour|minute|second|lane|class|speed|num_axles| gvw|wheelbase| wt1|spc1| wt2|spc2| wt3|spc3| wt4|spc4| wt5|spc5| wt6|spc6| wt7|spc7| wt8|spc8| wt9|spc9|wt10|spc10|wt11|spc11|wt12|spc12|wt13|spc13|wt14|spc14|wt15|spc15|wt16|spc16|wt17|dirtravel|yearmonth|exceed_80000|exceed_22400|Ticket|check_spacing|
+---------+----+-----+---+----+------+------+----+-----+-----+---------+----+---------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------------+------+-------------+
|       QB|  23|   

In [99]:
spc_2axle_loading_case_count_19_23 = df4.filter("check_spacing == 1").count()

spc_2axle_loading_case_count_19_23
#26,348 vehicles that have at least 1 pair of axles with spacing 1.34 < s < 3.05

26348

In [100]:
df4.createOrReplaceTempView("table4")

In [101]:
pair_conditions = " OR ".join([f"(wt{i} + wt{i+1} > 36000)" for i in range(1, 15)])
pair_conditions2 = " OR ".join([f"(wt{i} + wt{i+1} > 39600)" for i in range(1, 15)])

df5 = spark.sql(f"""
    SELECT * EXCEPT (Ticket),
        CASE 
            WHEN check_spacing = 1 AND ({pair_conditions}) THEN 1 
            ELSE 0 
        END AS 2axle_exceed_36000,
        CASE 
            WHEN check_spacing = 1 AND ({pair_conditions2}) THEN 1 
            ELSE Ticket 
        END AS Ticket
    FROM table4
""")

df5.show(5)

+---------+----+-----+---+----+------+------+----+-----+-----+---------+----+---------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------------+-------------+------------------+------+
|direction|year|month|day|hour|minute|second|lane|class|speed|num_axles| gvw|wheelbase| wt1|spc1| wt2|spc2| wt3|spc3| wt4|spc4| wt5|spc5| wt6|spc6| wt7|spc7| wt8|spc8| wt9|spc9|wt10|spc10|wt11|spc11|wt12|spc12|wt13|spc13|wt14|spc14|wt15|spc15|wt16|spc16|wt17|dirtravel|yearmonth|exceed_80000|exceed_22400|check_spacing|2axle_exceed_36000|Ticket|
+---------+----+-----+---+----+------+------+----+-----+-----+---------+----+---------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------------+---

In [102]:
exceed_2axle_limit_19_23 = df5.filter("2axle_exceed_36000 ==  1").count()

exceed_2axle_limit_19_23
#559 vehicles exceeded load req on at least 1 pair of axles with load > 36,000

559

In [103]:
exceed_2axle_limit_19_23_ticket_count = df5.filter("Ticket ==  1").count() - exceed_1axle_limit_19_23_ticket_count - exceed_gvw_limit_19_23_ticket_count

exceed_2axle_limit_19_23_ticket_count
#294,940 total tickets
#205,233 Tickets issued for GVW > 88000 lb
#89,659 Tickets issued for single axle overload
#48 Tickets issued for 2axle load  > 39,600 lb

48

In [104]:
df5.createOrReplaceTempView("table5")

In [105]:
#Creating compliant col , compliant = 1 non-compliant = 0
df6 = spark.sql(
    """ 
    SELECT *,
        CASE 
            WHEN exceed_80000 == 1 OR exceed_22400 == 1 OR 2axle_exceed_36000 == 1 THEN 0
            ELSE 1
        END AS compliant
    FROM table5
    """
)

df6.show(5)

+---------+----+-----+---+----+------+------+----+-----+-----+---------+----+---------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------------+-------------+------------------+------+---------+
|direction|year|month|day|hour|minute|second|lane|class|speed|num_axles| gvw|wheelbase| wt1|spc1| wt2|spc2| wt3|spc3| wt4|spc4| wt5|spc5| wt6|spc6| wt7|spc7| wt8|spc8| wt9|spc9|wt10|spc10|wt11|spc11|wt12|spc12|wt13|spc13|wt14|spc14|wt15|spc15|wt16|spc16|wt17|dirtravel|yearmonth|exceed_80000|exceed_22400|check_spacing|2axle_exceed_36000|Ticket|compliant|
+---------+----+-----+---+----+------+------+----+-----+-----+---------+----+---------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+---------

In [106]:
compliant_count_19_23 = df6.filter("compliant == 1").count()

compliant_count_19_23
#67,405,105 compliant vehicles

67405105

In [107]:
non_compliant_count_19_23 = df6.filter("compliant == 0").count()

non_compliant_count_19_23
#630,853 non-compliant vehicles

630853

In [108]:
spark.stop() #Dont forget to end session

Present (2024-2025)

In [109]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Capstone_Data") \
    .master("local[*]") \
    .getOrCreate()
print("Spark is running. Don't forget to stop session to terminate background java process!")

Spark is running. Don't forget to stop session to terminate background java process!


In [110]:
df = spark.read.csv("Data\Raw_Data\Weigh_in_Motion_24_25.csv", header=True, inferSchema=True)
df.show(5)

<>:1: SyntaxWarning: invalid escape sequence '\R'
<>:1: SyntaxWarning: invalid escape sequence '\R'
C:\Users\rubie\AppData\Local\Temp\ipykernel_9804\3273418509.py:1: SyntaxWarning: invalid escape sequence '\R'
  df = spark.read.csv("Data\Raw_Data\Weigh_in_Motion_24_25.csv", header=True, inferSchema=True)


+---------+----+-----+---+----+------+------+----+-----+-----+---------+-----+---------+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+
|direction|year|month|day|hour|minute|second|lane|class|speed|num_axles|  gvw|wheelbase| wt1|spc1|  wt2|spc2| wt3|spc3| wt4|spc4| wt5|spc5| wt6|spc6| wt7|spc7| wt8|spc8| wt9|spc9|wt10|spc10|wt11|spc11|wt12|spc12|wt13|spc13|wt14|spc14|wt15|spc15|wt16|spc16|wt17|dirtravel|yearmonth|
+---------+----+-----+---+----+------+------+----+-----+-----+---------+-----+---------+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+
|       QB|  24|    8| 26|   6|    30|    55|   2|    2|   52|        2| 4400|      9.4|2600| 9.4| 1800|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|NULL|

In [111]:
df.createOrReplaceTempView("table1")

In [112]:
#New column "exceed_80000" 1->exceeds 0-> doesnt, check if GVW exceed 80,000 lbs
#If gvw 10% above limit then ticket is issued , marked as 1 in Ticket col
df2 = spark.sql(
    f"""
        SELECT *, 
            CASE
                WHEN gvw > 80000 THEN 1
                ELSE 0
            END AS exceed_80000,
            CASE
                WHEN gvw > 88000 THEN 1
                ELSE 0
            END AS Ticket
    FROM table1
    WHERE direction = 'QB'
    """
)

df2.show(5)

+---------+----+-----+---+----+------+------+----+-----+-----+---------+-----+---------+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------+
|direction|year|month|day|hour|minute|second|lane|class|speed|num_axles|  gvw|wheelbase| wt1|spc1|  wt2|spc2| wt3|spc3| wt4|spc4| wt5|spc5| wt6|spc6| wt7|spc7| wt8|spc8| wt9|spc9|wt10|spc10|wt11|spc11|wt12|spc12|wt13|spc13|wt14|spc14|wt15|spc15|wt16|spc16|wt17|dirtravel|yearmonth|exceed_80000|Ticket|
+---------+----+-----+---+----+------+------+----+-----+-----+---------+-----+---------+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------+
|       QB|  24|    8| 26|   6|    30|    55|   2|    2|   52|        2| 4400|      9.4|2600| 

In [113]:
exceed_gvw_limit_24_25 = df2.filter("exceed_80000 ==  1").count()

exceed_gvw_limit_24_25
#26,416 exceed 80,000 lb limit from 2024-2025

26416

In [114]:
exceed_gvw_limit_24_25_ticket_count = df2.filter("Ticket ==  1").count()

exceed_gvw_limit_24_25_ticket_count
#4,951 Tickets issued for GVW > 88000 lb

4951

In [115]:
df2.createOrReplaceTempView("table2")

In [116]:
#If any single axle load is > 22400 lb weight is non-compliant marked as 1 in col: exceed_22400
#If any single axle load is > 24,640 lb col Ticket = 1

df3 = spark.sql(
    """ 
        SELECT * EXCEPT (Ticket), 
            CASE
                WHEN GREATEST(
                    CAST(wt1 AS DOUBLE), CAST(wt2 AS DOUBLE), CAST(wt3 AS DOUBLE), CAST(wt4 AS DOUBLE),
                    CAST(wt5 AS DOUBLE), CAST(wt6 AS DOUBLE), CAST(wt7 AS DOUBLE), CAST(wt8 AS DOUBLE),
                    CAST(wt9 AS DOUBLE), CAST(wt10 AS DOUBLE), CAST(wt11 AS DOUBLE), CAST(wt12 AS DOUBLE),
                    CAST(wt13 AS DOUBLE), CAST(wt14 AS DOUBLE), CAST(wt15 AS DOUBLE), CAST(wt16 AS DOUBLE),
                    CAST(wt17 AS DOUBLE)
                ) > 22400 THEN 1
                ELSE 0
            END AS exceed_22400,
            CASE
                WHEN GREATEST(
                    CAST(wt1 AS DOUBLE), CAST(wt2 AS DOUBLE), CAST(wt3 AS DOUBLE), CAST(wt4 AS DOUBLE),
                    CAST(wt5 AS DOUBLE), CAST(wt6 AS DOUBLE), CAST(wt7 AS DOUBLE), CAST(wt8 AS DOUBLE),
                    CAST(wt9 AS DOUBLE), CAST(wt10 AS DOUBLE), CAST(wt11 AS DOUBLE), CAST(wt12 AS DOUBLE),
                    CAST(wt13 AS DOUBLE), CAST(wt14 AS DOUBLE), CAST(wt15 AS DOUBLE), CAST(wt16 AS DOUBLE),
                    CAST(wt17 AS DOUBLE)
                ) > 24640 THEN 1
                ELSE Ticket  
            END AS Ticket
        FROM table2
        WHERE direction = 'QB'
    """
)



df3.show(5)

+---------+----+-----+---+----+------+------+----+-----+-----+---------+-----+---------+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------------+------+
|direction|year|month|day|hour|minute|second|lane|class|speed|num_axles|  gvw|wheelbase| wt1|spc1|  wt2|spc2| wt3|spc3| wt4|spc4| wt5|spc5| wt6|spc6| wt7|spc7| wt8|spc8| wt9|spc9|wt10|spc10|wt11|spc11|wt12|spc12|wt13|spc13|wt14|spc14|wt15|spc15|wt16|spc16|wt17|dirtravel|yearmonth|exceed_80000|exceed_22400|Ticket|
+---------+----+-----+---+----+------+------+----+-----+-----+---------+-----+---------+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------------+------+
|       QB|  24|    8| 26|   6|    30|    55|   2|    2

In [117]:
exceed_1axle_limit_24_25 = df3.filter("exceed_22400 ==  1").count() 

exceed_1axle_limit_24_25
#32,069 exceed 22400 lb one axle wt limit from 2024-2025

32069

In [118]:
exceed_1axle_limit_24_25_ticket_count = df3.filter("Ticket ==  1").count() - exceed_gvw_limit_24_25_ticket_count

exceed_1axle_limit_24_25_ticket_count 
#4,951 Tickets issued for GVW > 88000 lb
#15,089 Tickets issued for single axle overload

15089

In [119]:
df3.createOrReplaceTempView("table3")

In [120]:
#data spacing units are in meters
#Spacing requirement made for any two axles with  54" < spacing < 120"  approx 1.37 meters < s < 3.05 meters

spc_conditions = " OR ".join([f"(spc{i} >= 1.37 AND spc{i} <= 3.05)" for i in range(1, 17)])

df4 = spark.sql(f"""
    SELECT *,
        CASE 
            WHEN {spc_conditions} THEN 1 
            ELSE 0 
        END AS check_spacing
    FROM table3
    WHERE direction = 'QB'
""")

df4.show(5)


+---------+----+-----+---+----+------+------+----+-----+-----+---------+-----+---------+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------------+------+-------------+
|direction|year|month|day|hour|minute|second|lane|class|speed|num_axles|  gvw|wheelbase| wt1|spc1|  wt2|spc2| wt3|spc3| wt4|spc4| wt5|spc5| wt6|spc6| wt7|spc7| wt8|spc8| wt9|spc9|wt10|spc10|wt11|spc11|wt12|spc12|wt13|spc13|wt14|spc14|wt15|spc15|wt16|spc16|wt17|dirtravel|yearmonth|exceed_80000|exceed_22400|Ticket|check_spacing|
+---------+----+-----+---+----+------+------+----+-----+-----+---------+-----+---------+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------------+------+-------------+
|       QB|  

In [121]:
spc_2axle_loading_case_count_24_25 = df4.filter("check_spacing == 1").count()

spc_2axle_loading_case_count_24_25
#25,456 vehicles that have at least 1 pair of axles with spacing 1.34 < s < 3.05

25456

In [122]:
df4.createOrReplaceTempView("table4")

In [123]:
#Some of the wts were type str, casted them to int
pair_conditions = " OR ".join([f"(CAST(wt{i} AS DOUBLE) + CAST(wt{i+1} AS DOUBLE) > 36000)" for i in range(1, 15)])
pair_conditions2 = " OR ".join([f"(CAST(wt{i} AS DOUBLE) + CAST(wt{i+1} AS DOUBLE) > 39600)" for i in range(1, 15)])

df5 = spark.sql(f"""
    SELECT * EXCEPT (Ticket),
        CASE 
            WHEN check_spacing = 1 AND ({pair_conditions}) THEN 1 
            ELSE 0 
        END AS 2axle_exceed_36000,
        CASE 
            WHEN check_spacing = 1 AND ({pair_conditions2}) THEN 1 
            ELSE Ticket 
        END AS Ticket
    FROM table4
""")

df5.show(5)

+---------+----+-----+---+----+------+------+----+-----+-----+---------+-----+---------+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------------+-------------+------------------+------+
|direction|year|month|day|hour|minute|second|lane|class|speed|num_axles|  gvw|wheelbase| wt1|spc1|  wt2|spc2| wt3|spc3| wt4|spc4| wt5|spc5| wt6|spc6| wt7|spc7| wt8|spc8| wt9|spc9|wt10|spc10|wt11|spc11|wt12|spc12|wt13|spc13|wt14|spc14|wt15|spc15|wt16|spc16|wt17|dirtravel|yearmonth|exceed_80000|exceed_22400|check_spacing|2axle_exceed_36000|Ticket|
+---------+----+-----+---+----+------+------+----+-----+-----+---------+-----+---------+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+----------

In [124]:
exceed_2axle_limit_24_25 = df5.filter("2axle_exceed_36000 ==  1").count()

exceed_2axle_limit_24_25
#381 vehicles exceeded load req on at least 1 pair of axles with load > 36,000

381

In [125]:
exceed_2axle_limit_24_25_ticket_count = df5.filter("Ticket ==  1").count() - exceed_gvw_limit_24_25_ticket_count - exceed_1axle_limit_24_25_ticket_count

exceed_2axle_limit_24_25_ticket_count
#20,081 total tickets
#4,951 Tickets issued for GVW > 88000 lb
#15,089 Tickets issued for single axle overload
#41 Tickets issued for 2axle load  > 39,600 lb

41

In [126]:
df5.createOrReplaceTempView("table5")

In [127]:
#Creating compliant col , compliant = 1 non-compliant = 0
df6 = spark.sql(
    """ 
    SELECT *,
        CASE 
            WHEN exceed_80000 == 1 OR exceed_22400 == 1 OR 2axle_exceed_36000 == 1 THEN 0
            ELSE 1
        END AS compliant
    FROM table5
    """
)

df6.show(5)

+---------+----+-----+---+----+------+------+----+-----+-----+---------+-----+---------+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+------------+------------+-------------+------------------+------+---------+
|direction|year|month|day|hour|minute|second|lane|class|speed|num_axles|  gvw|wheelbase| wt1|spc1|  wt2|spc2| wt3|spc3| wt4|spc4| wt5|spc5| wt6|spc6| wt7|spc7| wt8|spc8| wt9|spc9|wt10|spc10|wt11|spc11|wt12|spc12|wt13|spc13|wt14|spc14|wt15|spc15|wt16|spc16|wt17|dirtravel|yearmonth|exceed_80000|exceed_22400|check_spacing|2axle_exceed_36000|Ticket|compliant|
+---------+----+-----+---+----+------+------+----+-----+-----+---------+-----+---------+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+-----+----+---------+---------+---

In [128]:
compliant_count_24_25 = df6.filter("compliant == 1").count()

compliant_count_24_25
#14,277,642 compliant vehicles

14277642

In [129]:
non_compliant_count_24_25 = df6.filter("compliant == 0").count()

non_compliant_count_24_25
#53,192 non-compliant vehicles

53192

In [130]:
spark.stop() #Dont forget to end session